# Task 1
This is the solution for QOSF cohort 11 mentee screening task 1.

The code below calls the global optimizer, differential evolution, with the different 
cost functions. Since the optimization functions may not give an exact match, the 
equivalence tests provide a final confirmation whether the resulting circuit matches a
Toffoli gate.

For steps followed while building this project, please see the [README.md](./README.md)

In [1]:
from parameter_tuning import (
    optimize_with_counts,
    optimize_with_frobenius,
    optimize_with_trace_norm,
    optimize_with_trace_overlap,
)
from equivalency import state_vector_equiv, unitaries_allclose, operator_equiv
from generate_circuits import generate_task_circuit, generate_toffoli
from fractions import Fraction
from typing import Union

import numpy as np

toffoli_circuit = generate_toffoli()

def pretty_pi_print(angle:float) -> str:
    if angle == 0:
        return f"{angle}"
    if angle == 1:
        return "pi"
    if angle == -1:
        return "-pi"
    return f"{angle} pi"

def process_results(
    results_times: Union[
        tuple[list[float], float, str], list[tuple[list[float], float, str]]
    ],
):
    if isinstance(results_times, tuple):
        results_times= [results_times]

    for result, time, cost_func in results_times:
        result_circuit = generate_task_circuit(*result)
        clean_angles = [
            Fraction(angle / np.pi).limit_denominator(32) for angle in result
        ]
        operator_equiv_test =  operator_equiv(result_circuit.copy(), toffoli_circuit.copy())
        state_vector_test =  state_vector_equiv(result_circuit.copy(), toffoli_circuit.copy())
        allclose_test =  unitaries_allclose(result_circuit.copy(), toffoli_circuit.copy())
        if (
            state_vector_test
            and allclose_test
            and operator_equiv_test
        ):
            print(
                f"Angles successfully found with {cost_func} in {time} seconds!\n"
                f"U3({pretty_pi_print(clean_angles[0])}, {pretty_pi_print(clean_angles[1])}, {pretty_pi_print(clean_angles[2])})\n"
                f"U3({pretty_pi_print(clean_angles[3])}, {pretty_pi_print(clean_angles[4])}, {pretty_pi_print(clean_angles[5])})"
            )
        else:
            print(
                f"Cost function {cost_func} failed to find an exact match after {time} seconds.\n"
                f"state_vector:{state_vector_test} - allclose:{allclose_test} - operator:{operator_equiv_test}"
                f"The best angles found were:\n"
                f"U3({pretty_pi_print(clean_angles[0])}, {pretty_pi_print(clean_angles[1])}, {pretty_pi_print(clean_angles[2])})\n"
                f"U3({pretty_pi_print(clean_angles[3])}, {pretty_pi_print(clean_angles[4])}, {pretty_pi_print(clean_angles[5])})"
            )


In [2]:
overlap_results = optimize_with_trace_overlap()
process_results((*overlap_results, "trace_overlap_cost"))

Angles successfully found with trace_overlap_cost in 26.460007905960083 seconds!
U3(-1/2 pi, -pi, 0)
U3(0, -1/4 pi, 0)


In [3]:
frobenius_results = optimize_with_frobenius()
process_results((*frobenius_results, "frobenius_norm"))

Angles successfully found with frobenius_norm in 95.35844302177429 seconds!
U3(-1/2 pi, -pi, 0)
U3(0, -2/11 pi, -2/29 pi)


In [4]:
trace_results = optimize_with_trace_norm()
process_results((*trace_results, "trace_norm"))

Angles successfully found with trace_norm in 85.04883122444153 seconds!
U3(-1/2 pi, -pi, 0)
U3(0, -2/11 pi, -1/15 pi)


In [ ]:
count_results = optimize_with_counts()
process_results((*count_results, "count_comparison"))